In [ ]:
"""
Created on Fri Jan 27 15:18 2023

Look at the results of the test of the NN on Smith data

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

DEFINE OPTIONS

In [ ]:
mod_size =  'medium' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'allbutconstants' #'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

In [ ]:
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

In [ ]:
melt_1D_bi646 = xr.open_dataset(outputpath_melt_nn + 'SMITH_bi646/evalmetrics_1D_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_allyy_bi646.nc')
melt_1D_bf663 = xr.open_dataset(outputpath_melt_nn + 'SMITH_bf663/evalmetrics_1D_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_allyy_bf663.nc')


FIRST IMPRESSION - RMSE

In [ ]:
np.sqrt((((melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='Gt'))**2).mean())

In [ ]:
np.sqrt((((melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='box1'))**2).mean())

In [ ]:
np.sqrt((((melt_1D_bf663['predicted_melt'] - melt_1D_bf663['reference_melt']).sel(metrics='Gt'))**2).mean())

In [ ]:
np.sqrt((((melt_1D_bf663['predicted_melt'] - melt_1D_bf663['reference_melt']).sel(metrics='box1'))**2).mean())

LOOK INTO MORE DETAILS

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'SMITH_bi646/nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy01.nc').drop('time')
file_isf = file_isf_orig.sel(Nisf=isf_dim)

In [ ]:
def plot_scatter_all_isf(param_melt_tuned, target_Gt_yr, file_isf):
    
    regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
    colors = ['deepskyblue','brown','red','orange','limegreen','seagreen','magenta']
    symbol = ['o','v','>','p','*','s','<','^','X','d']
    
    f = plt.figure()
    f.set_size_inches(8.25*1.5, 8.25*1.5)

    ax={}

    i = 0
    nn=0
    ii=0
    marker_symbol = np.zeros((len(regions))).astype(int)
    for rr,reg in enumerate(regions):
        k = 0
        marker_color = colors[rr]
        subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
        for kisf in tqdm(subset_isf.Nisf):
            marker_type = symbol[k]

        
            #print(i)

            x_axis = param_melt_tuned.sel(Nisf=kisf)
            y_axis = target_Gt_yr.sel(Nisf=kisf)


            ax[i] = f.add_subplot(6,6,i+1)

            ax[i].scatter(x_axis, 
                        y_axis,
                        s=10, c=marker_color, marker=marker_type, edgecolors='None',
                        rasterized=True)
            a = 0
            a = 0

            
            min_xy = min(x_axis.min(),y_axis.min())
            max_xy = max(x_axis.max(),y_axis.max())
            onetooneline = np.arange(min_xy,max_xy)
            ax[i].set_xlim(min_xy-5,max_xy+5)
            ax[i].set_ylim(min_xy-5,max_xy+5)
            ax[i].plot(onetooneline,onetooneline,'k-')
            #if i == 0 or i==3:
            #    ax[i].set_ylabel('Target melt [Gt/yr]')
            #if i == 3 or i==4:
            #    ax[i].set_xlabel('Predicted melt [Gt/yr]')



            ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))

            i = i+1
            k = k+1


    f.tight_layout()
    sns.despine()

    return f

In [ ]:
plot_scatter_all_isf(melt_1D_bi646['predicted_melt'].sel(metrics='Gt'), melt_1D_bi646['reference_melt'].sel(metrics='Gt'), file_isf)

In [ ]:
plot_scatter_all_isf(melt_1D_bf663['predicted_melt'].sel(metrics='Gt'), melt_1D_bf663['reference_melt'].sel(metrics='Gt'), file_isf)